In [1]:
%pip install --upgrade --quiet  langchain-google-genai
%pip install python-dotenv --quiet
%pip install --upgrade --quiet  langchain pandas fpdf

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [32]:
import os
import sys
import json
from langchain_google_genai import GoogleGenerativeAI
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import pandas as pd
from fpdf import FPDF
load_dotenv()

True

In [33]:
KEY = os.getenv("GOOGLE_API_KEY")

In [34]:
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=KEY,temperature=.7)


In [35]:
print(
    llm.invoke(
        "What are some of the pros and cons of Python as a programming language?"
    )
)

**Pros of Python:**

* **Simplicity and Readability:** Python is known for its simple and intuitive syntax, making it easy to learn and write code. Its code structure resembles natural language, making it highly readable.
* **Versatility:** Python is a versatile language that can be used in various domains, including web development, data science, machine learning, and scripting.
* **Extensive Libraries:** Python has a vast collection of libraries and frameworks that provide pre-built modules for common tasks, such as data manipulation, machine learning, and web development.
* **Cross-Platform Compatibility:** Python code can run on multiple operating systems (Windows, macOS, Linux) without the need for recompilation.
* **Beginner-Friendly:** Python's simplicity makes it an excellent choice for beginners who are starting their programming journey.
* **Community Support:** Python has a large and active community of developers who provide support, documentation, and resources.

**Cons of

***design of response***

In [36]:
RESPONSE_JSON = {
    "1":{
        "mcq": "question",
        "options":{
            "a": "choice",
            "b": "choice",
            "c": "choice",
            "d": "choice"
        },
        "correct": "correct_answer"
    },
    "2":{
        "mcq": "question",
        "options":{
            "a": "choice",
            "b": "choice",
            "c": "choice",
            "d": "choice"
        },
        "correct": "correct_answer"
    },
    "3":{
        "mcq": "question",
        "options":{
            "a": "choice",
            "b": "choice",
            "c": "choice",
            "d": "choice"
        },
        "correct": "correct_answer"
    },
    "4":{
        "mcq": "question",
        "options":{
            "a": "choice",
            "b": "choice",
            "c": "choice",
            "d": "choice"
        },
        "correct": "correct_answer"
    },
}

In [37]:
TEMPLATE_1='''
   TEXT: {text}
You are an expert in creating multiple choice questions. Given the above text, it is your job to create a quiz of {number} questions for {subject} students in a {tone} tone. Ensure the questions are unique and relevant to the text.
Use the format provided below as a guide to construct your quiz, making sure to include {number} MCQs:

{response_json}

'''

In [38]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE_1
)

In [39]:
quiz_chain = LLMChain(
    llm=llm,
    prompt = quiz_generation_prompt,
    output_key = "quiz",
    verbose=True
)

In [40]:
TEMPLATE_2 = '''
    You are expert in english gramer and writer. Give a muitiple choice for students in subject {subject}. \
    You need to evaluate the complexity of question and give a complete ananylsis of quiz. Only use at max 50 words for each.\
    Quiz_MCQs:
    {quiz}

    check from expert English writer of the above quiz:
'''

In [41]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject","quiz"],
    template=TEMPLATE_2
)

In [42]:
review_chain = LLMChain(
    llm=llm,
    prompt = quiz_evaluation_prompt,
    output_key = "review",
    verbose=True
)

In [43]:
generate_evalution_chain = SequentialChain(
    chains=[quiz_chain,review_chain],
    input_variables=["text","number","subject","tone","response_json"],
    output_variables=["quiz","review"],
    verbose=True
)

In [44]:
file_path = "/Users/naveen/Developer/GenAI/quiz_generator/data.txt"

In [45]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [46]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "question", "options": {"a": "choice", "b": "choice", "c": "choice", "d": "choice"}, "correct": "correct_answer"}, "2": {"mcq": "question", "options": {"a": "choice", "b": "choice", "c": "choice", "d": "choice"}, "correct": "correct_answer"}, "3": {"mcq": "question", "options": {"a": "choice", "b": "choice", "c": "choice", "d": "choice"}, "correct": "correct_answer"}, "4": {"mcq": "question", "options": {"a": "choice", "b": "choice", "c": "choice", "d": "choice"}, "correct": "correct_answer"}}'

In [47]:
NUMBER = 10
SUBJECT = "deep Learning",
TONE = "simple"

In [70]:
res = generate_evalution_chain.__call__(
        {
        "text":TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

   TEXT: Collaborative filtering (CF) is a successful approach commonly used by many recommender systems.
Conventional CF-based methods use the ratings given to items by users as the sole source of information for learning to make recommendation. 
However, the ratings are often very sparse in many applications, causing CF-based methods to degrade significantly in their recommendation performance. 
To address this sparsity problem, auxiliary information such as item content information may be utilized. Collaborative topic regression (CTR) is an appealing recent method taking this approach which tightly couples the two components that learn from two different sources of information. 
Nevertheless, the latent representation learned by CTR may not be very effective when the auxiliary information is very sparse. 
To address this problem, we generalize recently advances in deep learning fro

In [71]:
res

{'text': 'Collaborative filtering (CF) is a successful approach commonly used by many recommender systems.\nConventional CF-based methods use the ratings given to items by users as the sole source of information for learning to make recommendation. \nHowever, the ratings are often very sparse in many applications, causing CF-based methods to degrade significantly in their recommendation performance. \nTo address this sparsity problem, auxiliary information such as item content information may be utilized. Collaborative topic regression (CTR) is an appealing recent method taking this approach which tightly couples the two components that learn from two different sources of information. \nNevertheless, the latent representation learned by CTR may not be very effective when the auxiliary information is very sparse. \nTo address this problem, we generalize recently advances in deep learning from i.i.d. input to non-i.i.d. (CF-based) input and propose in this paper a hierarchical Bayesian m

In [72]:
quiz = res.get('quiz')

In [73]:
quiz2 = json.loads(quiz)


In [74]:
type(quiz2)

dict

In [75]:
data = quiz2

In [22]:
quiz_data = []
for key,value in quiz.items():
    mcq = value.get('mcq')
    options = [f'{key}: {val}' for key, val in value.get('options').items()]
    correct = value.get('correct')
    quiz_data.append({'MCQ':mcq,"options":options, "Correct Answer": correct})

In [23]:
quiz_data

[{'MCQ': 'What is the primary drawback of conventional Collaborative Filtering (CF)-based methods?',
  'options': ['a: Overfitting',
   'b: Sparsity of ratings',
   'c: High computational cost',
   'd: Lack of user personalization'],
  'Correct Answer': 'b'},
 {'MCQ': 'What type of information is utilized by Collaborative Topic Regression (CTR) to address the sparsity problem?',
  'options': ['a: User demographics',
   'b: Item content',
   'c: Social network connections',
   'd: Historical purchase data'],
  'Correct Answer': 'b'},
 {'MCQ': 'What is the main limitation of CTR when auxiliary information is sparse?',
  'options': ['a: Increased overfitting',
   'b: Ineffective latent representation',
   'c: Poor convergence',
   'd: Difficulty in model training'],
  'Correct Answer': 'b'},
 {'MCQ': 'In Collaborative Deep Learning (CDL), what type of representation learning is performed for content information?',
  'options': ['a: Linear regression',
   'b: Deep representation learning',

In [24]:
pd.DataFrame(quiz_data)

,MCQ,options,Correct Answer
0,What is the primary drawback of conventional C...,"[a: Overfitting, b: Sparsity of ratings, c: Hi...",b
1,What type of information is utilized by Collab...,"[a: User demographics, b: Item content, c: Soc...",b
2,What is the main limitation of CTR when auxili...,"[a: Increased overfitting, b: Ineffective late...",b
3,"In Collaborative Deep Learning (CDL), what typ...","[a: Linear regression, b: Deep representation ...",b
4,What is the main advantage of CDL over CTR?,[a: Improved handling of sparse auxiliary info...,a
5,What type of input does CDL generalize recentl...,"[a: i.i.d., b: Non-i.i.d., c: Structured, d: U...",b
6,What is the purpose of the feedback matrix in ...,"[a: To store user ratings, b: To represent ite...",a
7,Which of the following is NOT a benefit of usi...,"[a: Enhanced recommendation performance, b: Re...",c
8,In which domains have experiments shown that C...,"[a: E-commerce, b: Movie recommendation, c: Mu...",d
9,What is the main idea behind CDL?,[a: Combining deep learning and collaborative ...,a


In [25]:
df = pd.DataFrame(quiz_data)

In [78]:
class MCQPDF(FPDF):
    def __init__(self, include_answers=True):
        super().__init__()
        self.include_answers = include_answers
        self.title_added = False
        self.add_page()
        self.set_auto_page_break(auto=True, margin=30)

    def header(self):
        title = 'MCQ Assessment - Answers Included' if self.include_answers else 'MCQ Assessment - For Students'
        if not self.title_added:
            self.set_font('Arial', 'I', 16)
            self.cell(0, 10, title, 0, 1, 'C')
            self.cell(0, 10, '', 0, 1, 'C')
            self.title_added = True

    def footer(self):

        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, 'Generated by Hana-AI', 0, 0, 'L')
        self.cell(0, 10, 'Page %s' % self.page_no(), 0, 0, 'C')

    def create_document(self,data):
        self.set_font("Arial", size=12)
        for key, value in data.items():
            self.set_font("Arial", 'B', 12)
            self.multi_cell(0, 10, f"Question {key}: {value['mcq']}")
            self.set_font("Arial", size=12)
            for opt_key, opt_value in value['options'].items():
                self.multi_cell(0, 10, f"{opt_key}: {opt_value}")
            if self.include_answers:
                self.multi_cell(0, 10, f"Correct Answer: {value['correct']}")
            self.ln(10)


In [79]:
pdf_with_answers = MCQPDF(include_answers=True)
pdf_with_answers.set_author("Hana-AI")
pdf_with_answers.create_document(data)
pdf_with_answers.output("MCQs_by_HanaAi_with_Answers.pdf")

pdf_without_answers = MCQPDF(include_answers=False)
pdf_without_answers.set_author("Hana-AI")
pdf_without_answers.create_document(data)
pdf_without_answers.output("MCQs_by_HanaAi_.pdf")

''

In [28]:
print("fucked successfully....")

fucked successfully....
